# Apriori algorithm

In [1]:
from collections import defaultdict
from itertools import combinations, chain
from decimal import Decimal, getcontext, ROUND_HALF_UP

In [2]:
support = .05

In [3]:
getcontext().rounding=ROUND_HALF_UP

In [4]:
def rounding(value, k):
    return type(value)(round(Decimal(value), k))

In [15]:
def apriori(data, support):
    candidates = {frozenset([x]) for x in set(chain(*data))}
    result = dict()
    k = 2
    def scan():
        count = defaultdict(int)
        for transaction in data:
            for candidate in candidates:
                if candidate.issubset(transaction):
                    count[candidate] += 1
        return {k: v/len(data) for k, v in count.items() if float(v)/len(data) >= support}
    while candidates:
        filtered = scan()
        result[k - 1] = filtered
        candidates = {i.union(j) for i in filtered for j in filtered if len(i.union(j)) == k}
        k += 1
    return result

In [16]:
def define(freq, transactions, confidence=.0):
    def f(item): return freq[len(item)][item]
    for k, v in freq.items():
        if k == 1: continue
        for item in v:
            for element in map(frozenset, chain(*[combinations(item, i) for i, e in enumerate(item, 1)])):
                remain = item.difference(element)
                if remain:
                    (a, b), (c, d) = f(item).as_integer_ratio(), f(element).as_integer_ratio()
                    conf = (a/b)/(c/d)
                    if conf >= confidence:
                        yield element, remain, rounding(f(item) * 100, 2), rounding(conf * 100, 2)

In [17]:
with open('input.txt') as f:
    data = [list(map(int, line.split())) for line in f.readlines()]

In [18]:
freq = apriori(data, .05)

In [19]:
rules = list(define(freq, data))

In [20]:
with open('output5.txt', 'w') as f:
    for item, asso, sup, conf in rules:
        f.write('{{{}}}\t{{{}}}\t{:.2f}\t{:.2f}\n'.format(','.join(map(str, item)), ','.join(map(str, asso)), sup, conf))